### Wetterdaten für die gewählten Stationen einlesen und bereinigen

- Pfade für die einzelnen Wetterdateien aus der von 04_daten_herunterladen erzeugten Datei datei_liste.txt einlesen
- Die 10 Dateien mit Wetterdaten einlesen
- Nicht relevante Spalten entfernen
- Datentyp für Spalten anpassen
- Daten auf den Zeitraum von 2014-2024 reduzieren
- Fehlerhafte Daten entfernen

In [1]:
# Datei datei_liste.txt einlesen
datei_liste <- readLines("datasets/orte/datei_liste.txt")

# Liste, um die Inhalte der Dateien zu speichern
wetterdaten <- list()

# Jede Datei aus der Liste einlesen
for (i in seq_along(datei_liste)) {

  # Der Pfad zur Datei
  dateipfad <- datei_liste[i]
  
  # Datei einlesen
  wetterdaten[[i]] <- read.table(dateipfad, 
    header = TRUE, 
    sep = ";", 
    fill = TRUE, 
    fileEncoding = "UTF-8")
  
  # Benenne das dataframe in der Liste nach der Stations ID
  name <- paste("station", wetterdaten[[i]]$STATIONS_ID[1], sep = "_")
  names(wetterdaten)[i] <- name

  # Nur relevante Spalten behalten
  wetterdaten[[i]] <- wetterdaten[[i]][, c("STATIONS_ID", "MESS_DATUM", "RSK", "SDK", "TMK")]

  # Datentypen anpassen
  wetterdaten[[i]]$MESS_DATUM <- as.Date(as.character(wetterdaten[[i]]$MESS_DATUM), format = "%Y%m%d")

  # Nur die letzten 10 Jahre behalten
  wetterdaten[[i]] <- tail(wetterdaten[[i]], 3652)
  
}

In [ ]:
# Entfernen von fehlerhaften Daten

for (i in seq_along(wetterdaten)) {
  wetterdaten[[i]] <- subset(wetterdaten[[i]], RSK <= 400.0 & RSK >= 0.0)
  wetterdaten[[i]] <- subset(wetterdaten[[i]], SDK <= 24 & SDK >= 0)
  wetterdaten[[i]] <- subset(wetterdaten[[i]], TMK <= 60 & TMK >= -60)
}

In [ ]:
# Die bereinigten Datensätze für die weitere Verwendung speichern

saveRDS(wetterdaten, "datasets/rds/wetterdaten.rds")